In [ ]:
'''
The following script provides a frame to build your own machine learning
model to automatically categorize your financial transaction.

It is not entirelly an out-of-box solution -
although if you prepare your data - see details below - you can 
just run this script to train your data and make predictions.
Further advantage is, it can make meaningful predictions 
even with a few thousand financial transactions
(at my test, it produced up to 83% accuracy with 20 categories and 6000 training examples)

Minimal Requirements: 
- python===3.7
- pandas==0.25.3
- numpy==1.18.1
- scikit-learn==0.22.1

Also you need some training (financial) data to train your own model
'''

In [1]:
# 1.) import modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import collections
import re
import random

In [12]:
# 2.)load datasets
# For demonstration, I make up an example here
# your data could look diferently, but try to phrase the columns similarly
data= {"Partnername":["POS Terminal.","Grocery Store","Your company","Your Company","Internet bill"],
      "Partner IBAN":["UK123400000567870001","","US282100063955027305","UK121300000567870001","UK234500000567870001"],
      "Partner BIC":["BKAUUSCC","","RJMWUSYYXXX","GIBAUSYYXXX","ZTXCUSYYXXX"],
      "Partner Account Number":[None,40200302900,None,None,None],
      "Partner Bank-Code (BLZ)":[11000,22000,33000,3300,4400],
      "Ammount":[-10,-19.19,-5.39,1877.44,-14.98],
      "Currency":["GBP","GBP","GBP","GBP","GBP"],
      "Booking information":["SB-Cash K2 S90380 21.02/17:10","GROCERY THANKS 2320W K2 18.03. 12:43","KV5350699","SALARY 2020/02","YOUR KT 4802175974 /0021047921/20200 228/GROSS"],
      "Payment reference":["201251005192BZX-003EDXXY6C93","309251005192BZX-003EDXSDHJA93","371287-DAJKSH-DSJAHJK","F00-BA332898R","T3Mp3818"],
      "Category":["Cash from ATM","Grocery","Purchase within workplace","Salary","Internet, telecommunication"]}

#The final column "Category" is especially vital!
#you have to take your training data an pre-categorize it manually
#(or by any other method) and append this information to the original
#dataset.

df_test_bank_data=pd.DataFrame(data)

#Obviously, you should load your own data by .csv or excel format. 
#For loading excel, uncomment the next 2 lines and replace with your file location
#excel_loc=r"/my_machine/Desktop/bank_export_2017-01-01_2020-03-19.xlsx"
#df_test_bank_data=pd.read_excel(excel_loc)

df_test_bank_data

,Partnername,Partner IBAN,Partner BIC,Partner Account Number,Partner Bank-Code (BLZ),Ammount,Currency,Booking information,Payment reference,Category
0,POS Terminal.,UK123400000567870001,BKAUUSCC,NaN,11000,-10.00,GBP,SB-Cash K2 S90380 21.02/17:10,201251005192BZX-003EDXXY6C93,Cash from ATM
1,Grocery Store,,,4.020030e+10,22000,-19.19,GBP,GROCERY THANKS 2320W K2 18.03. 12:43,309251005192BZX-003EDXSDHJA93,Grocery
2,Your company,US282100063955027305,RJMWUSYYXXX,NaN,33000,-5.39,GBP,KV5350699,371287-DAJKSH-DSJAHJK,Purchase within workplace
3,Your Company,UK121300000567870001,GIBAUSYYXXX,NaN,3300,1877.44,GBP,SALARY 2020/02,F00-BA332898R,Salary
4,Internet bill,UK234500000567870001,ZTXCUSYYXXX,NaN,4400,-14.98,GBP,YOUR KT 4802175974 /0021047921/20200 228/GROSS,T3Mp3818,"Internet, telecommunication"


In [6]:
df_test_bank_data["Partner Account Number"]

0               
1    40200302900
2               
3               
4               
Name: Partner Account Number, dtype: object

In [13]:
#3.) set functions for further data cleaning and data transformation

#This function can be applied to remove all numeric values from the "Booking information" column.
#We need these as "Booking Information" columns otherwise would contain only unique values
#By removing the numbers, we can categorize the transformed - but uglier - remaining values

def precategorization2(bookinfo):
    result = ''.join([i for i in bookinfo if not i.isdigit()])
    return result

#This function cleans the input dataframe,by filling empty values
#transforming mixed columns to integers (if possible)
#excluding non-relevant columns
#(if this function is used on a training dataset, then the Category column is included)
# also transforming the "Booking information" column by removing the non relevant digits from it
    
def clear_dataframes(input_dataframe,is_training=False):
    input_dataframe['Partnername']=input_dataframe['Partnername'].fillna("Unknown partner")
    input_dataframe["Partner IBAN"]=input_dataframe["Partner IBAN"].fillna("no IBAN")
    input_dataframe["Partner BIC"]=input_dataframe["Partner BIC"].fillna("no BIC")
    input_dataframe["Partner Account Number"]=input_dataframe["Partner Account Number"].fillna(0)
    input_dataframe["Partner Account Number"]=(np.round(input_dataframe["Partner Account Number"],2)).astype(int)
    input_dataframe['Partner Bank-Code (BLZ)']=input_dataframe['Partner Bank-Code (BLZ)'].fillna(0)
    input_dataframe['Partner Bank-Code (BLZ)']=(np.round(input_dataframe['Partner Bank-Code (BLZ)'],2)).astype(int)
    input_dataframe['Booking information']=input_dataframe['Booking information'].fillna("no Booking information available")
    if is_training==True:
        df_optimized_input=input_dataframe[['Partnername', 'Partner IBAN', 'Partner BIC','Partner Account Number', 'Partner Bank-Code (BLZ)', 'Ammount', 'Booking information',  'Category']]
    else:
        df_optimized_input=input_dataframe[['Partnername', 'Partner IBAN', 'Partner BIC','Partner Account Number', 'Partner Bank-Code (BLZ)', 'Ammount', 'Booking information']]
    df_optimized_input=df_optimized_input.rename(columns={'Partner IBAN':'Partner_IBAN',"Partner BIC":"Partner_BIC",'Partner Account Number':'Partner_Account_Number','Partner Bank-Code (BLZ)':'Partner_Bank_Code'})
    df_optimized_input['Booking information'] = df_optimized_input.apply(lambda x: precategorization2(x['Booking information']),axis=1)
    return df_optimized_input

#By numerically categorizing certain columns (like every "Grocery" as partner should belong to category 2)
# in the training dataset has the disadvantage, we will not know, what happens
# if a new value/cateogry appears in the test/new dataset.
# To solve this issue, the following function numeically 
# checks, if a category value was already set for a category.
# if yes, the original value will be used (e.g. "Grocery" will be 2 again)
# if not, then a new value will be generated
def valuifier(inputvars,orig_strs,orig_vals):
    finalvalue=[]
    for a in range(len(inputvars)):
        if inputvars[a] in orig_strs:
            minivar=orig_strs.index(inputvars[a])
            finalvalue.append(orig_vals[minivar])
        else:
            finalvalue.append(100+a)
    return finalvalue



#The predictor function will provide a numeric category value, like "4"
# which is not humanly readable. To solve this issue, the function
# finds looks up the original category valuable based on the 
# predicted number and returns the original string (e.g. 4 => "Salary")
def textifier(predicted_value,original_y_value,original_y_string):
    real_category=[]
    for a in range(len(predicted_value)):
        if predicted_value[a] in original_y_value:
            minivar=original_y_value.index(predicted_value[a])
        else:
            minivar=2
        real_category.append(original_y_string[minivar])
    return real_category

#A minor function, which checks, if a certain word is in the "Booking information" column

def wordsearcher(bookinfo,currentword):
    if currentword in bookinfo:
        return 1
    else:
        return 0

# This function can be used (optionally) to create new rows 
# in the training dataset, by copying the old ammounts and randomly
# adjusting the "Ammount" from the training dataset
    
def data_faker(ammount):
    final_amount=ammount*random.choice([0.9,1.1])
    return final_amount


In [15]:
#4.) clean training dataset
training_dataframe=clear_dataframes(df_test_bank_data,is_training=True)

#optional
#if you feel your training dataset is way to small and you would like to improve the training
#accuracy, you can uncomment the following sections for these
#unique_category_values=training_dataframe['Category'].unique().tolist()
#for b in range(2):
#    for a in range(len(unique_category_values)):
#        df_temp=training_dataframe[training_dataframe['Category']==unique_category_values[a]]
#        df_temp["Ammount"]=training_dataframe.apply(lambda x: data_faker(x['Ammount']),axis=1)
#        training_dataframe=training_dataframe.append(df_temp, ignore_index=True)

#the following section splits the column "Booking information" to "words" (by spaces),
# and remove the special characters. Then the 
# top 20 words (assuming there are as many, otherwise use all phrases)
# and create news columns from these words, whether these words are in the 'Booking information' column
# for example "contains Salary" is 0 or 1

wordlist=[]
list_of_booking_info=training_dataframe['Booking information'].tolist()
for a in range(len(list_of_booking_info)):
    wordlist.append(list_of_booking_info[a].split())
flatlist=[item for sublist in wordlist for item in sublist]
flatlist2=[]
for b in range(len(flatlist)): 
    c=re.sub('\,|\?|\.|\!|\/|\;|\:', '', flatlist[b])
    flatlist2.append(c)
flatlist2=filter(None, flatlist2)
ctr = collections.Counter(flatlist2)
if len(ctr)>20:
    listholder=ctr.most_common(20)
else:
    listholder=ctr.most_common(len(ctr))
for a in range(len(listholder)):
    training_dataframe["contains "+str(listholder[a][0])] = training_dataframe.apply(lambda x: wordsearcher(x['Booking information'],listholder[a][0]),axis=1)
training_dataframe["Ammount"]=training_dataframe["Ammount"].round(-1).astype(int)

# The following section splits the dataset to numerical values or strings (called object in pandas)
# then the label encoder check all the string columns and replace these with numerical values.
# Finally, these numeric values are merged together once again.
# We need these steps, as machine learning algorithms can usually handle numeric input values directly.

le = LabelEncoder()
int_df = training_dataframe.select_dtypes(include=['int64']).copy()
float_df=training_dataframe.select_dtypes(include=['float64']).copy()
df_int_float = pd.concat([float_df,int_df], axis=1)
obj_df = training_dataframe.select_dtypes(include=['object']).copy()
obj_df_trf=obj_df.astype(str).apply(le.fit_transform)
df_ready_to_split = pd.concat([df_int_float,obj_df_trf], axis=1)
# We have to remove the "Category" variable to a separate target variable
# to let our machine learning optimize the modell to it
y_var=df_ready_to_split.pop("Category")

# if you wish to split the training dataset to training and test set, uncomment the following line
# however, if you check your model accuracy with cross validation (see Step 5.)),
# it is not necessary to split the model.
#X_train, X_test, y_train, y_test = train_test_split(df_ready_to_split, y_var,test_size=0.1)

# This creates a dataframe from the transformed string column,
# where the original strings and their numeric representation is matched.
# Thus if something goes wrong, we can alway check this which value belongs
# to which numeric category.
df_recovery=obj_df.merge(obj_df_trf, left_index=True, right_index=True)    

In [ ]:
#5.) checking the model
# Assuming your input sample will be somewhat small (probably a few thousand transactions)
# it make sense to first make a cross validation 
# (sklearn makes multiple folds and compare the result on all training data!)
# Uncomment the following lines and see the mean accuracy score in percent
# If you find the result acceptable, go to the next section - otherwise you might need to provide
# more/better input for your model.
#
#cv_scores = cross_val_score(RandomForestClassifier(random_state=0), df_ready_to_split, y_var, 
#                            cv=5,
#                            scoring='accuracy')
#print(cv_scores.mean())


In [17]:
#6.) training the model
# we will use a simple Random Forest classifier on all of the training data
forest_model = RandomForestClassifier(random_state=0)
forest_model.fit(df_ready_to_split, y_var)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [18]:
#7.)load new data and make predictions
# For now, we will use the original training dateset, but you should load a new financial transactions to 
# make predicitons. Make sure, it has the same format, without a column "Category"
# (as it will predict these)
df_forecast_data=pd.DataFrame(data)
#cleaning new dataset
df_new_optimized_cols=clear_dataframes(df_forecast_data,is_training=False)
for a in range(len(listholder)):
    df_new_optimized_cols["contains "+str(listholder[a][0])] = df_new_optimized_cols.apply(lambda x: wordsearcher(x['Booking information'],listholder[a][0]),axis=1)
df_new_optimized_cols["Ammount"]=df_new_optimized_cols["Ammount"].round(-1).astype(int)
n_int_df = df_new_optimized_cols.select_dtypes(include=['int64']).copy()
n_float_df=df_new_optimized_cols.select_dtypes(include=['float64']).copy()
df_n_int_float = pd.concat([n_float_df,n_int_df], axis=1)
n_obj_df = df_new_optimized_cols.select_dtypes(include=['object']).copy()
#As the new dataset might or might not have such data in it, which was in the training dataset
# the following section checks the new value from the earlier encoding.
# If it finds the old data, it provides the same class.
# If not, it sets a new value for these.
feats=n_obj_df.columns.tolist()
for b in range(len(feats)):
    n_obj_df[feats[b]]=valuifier(n_obj_df[feats[b]].tolist(),df_recovery[str(feats[b])+"_x"].tolist(),df_recovery[str(feats[b])+"_y"].tolist())
testval=valuifier(n_obj_df["Partnername"].tolist(),df_recovery["Partnername_x"].tolist(),df_recovery["Partnername_y"].tolist())
n_obj_df["Partnername"]=testval
df_n_final = pd.concat([df_n_int_float,n_obj_df], axis=1)
# This scrpit predicts the new categories - but in numerical form
y_predicted_class=forest_model.predict(df_n_final)
# To make the result humanly readable, the following script
# recovers from the training "recovery" dataset the original classes
# and appends humanly readable string categories to the new dataset
y_predicted_str_class=textifier(y_predicted_class,df_recovery["Category_y"].tolist(),df_recovery["Category_x"].tolist())
df_new_optimized_cols["Predicted Category"]=y_predicted_str_class

In [19]:
# Hopefully you see now the right categories
df_new_optimized_cols

,Partnername,Partner_IBAN,Partner_BIC,Partner_Account_Number,Partner_Bank_Code,Ammount,Booking information,contains K,contains SB-Cash,contains S,contains GROCERY,contains THANKS,contains W,contains KV,contains SALARY,contains YOUR,contains KT,contains GROSS,Predicted Category
0,POS Terminal.,UK123400000567870001,BKAUUSCC,0,11000,-10,SB-Cash K S ./:,1,1,1,0,0,0,0,0,0,0,0,Cash from ATM
1,Grocery Store,,,40200302900,22000,-20,GROCERY THANKS W K .. :,1,0,1,1,1,1,0,0,0,0,0,Grocery
2,Your company,US282100063955027305,RJMWUSYYXXX,0,33000,-10,KV,1,0,0,0,0,0,1,0,0,0,0,Purchase within workplace
3,Your Company,UK121300000567870001,GIBAUSYYXXX,0,3300,1880,SALARY /,0,0,1,0,0,0,0,1,0,0,0,Salary
4,Internet bill,UK234500000567870001,ZTXCUSYYXXX,0,4400,-10,YOUR KT // /GROSS,1,0,1,0,0,0,0,0,1,1,1,"Internet, telecommunication"
